# Text Polarity Classification

## Import and preprocessing

In [1]:
# Connect drive
from google.colab import drive
drive.mount('/content/drive')

# Text preprocessing and imports
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

import pandas as pd
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import tensorflow as tf
import torch

# Finetuning

!pip install transformers
!pip install simpletransformers
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from simpletransformers.classification import ClassificationModel, ClassificationArgs

Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### GPU Colab

In [2]:
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

if torch.cuda.is_available(): 
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


### Upload training and test

In [3]:
train = pd.read_csv('/content/drive/MyDrive/NCU/Data Science and Machine Learning/Assignment 3/Datasets/train.csv')
train = train.drop(train.columns[0], axis=1)

In [4]:
valid = pd.read_csv('/content/drive/MyDrive/NCU/Data Science and Machine Learning/Assignment 3/Datasets/test.csv')
valid = valid.drop(valid.columns[0], axis=1)

### Preprocessing (stopwords and punctuation removal, lemmatizing)

In [5]:
def preprocess_text(text):
    # convert text to lowercase
    text = text.lower()
    # remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # tokenize the text into words
    words = nltk.word_tokenize(text)
    # remove stop words
    words = [word for word in words if word not in stopwords.words('english')]
    # lemmatize the remaining words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    # join the words back into a single string
    text = " ".join(words)
    return text

def text_valid_prepr(train_df, valid_df):

    print('Preprocessing the training data...')

    train_df['TEXT'] = train_df['TEXT'].apply(preprocess_text)

    print('Preprocessing the validation data...')
    valid_df = [sent for sent in valid_df['TEXT']]
  
    return train_df, valid_df

train, valid = text_valid_prepr(train, valid)

def max_length(train_df):

    train_list = [sent for sent in train['TEXT']]
    
    token_list = []
    max_length = 0

    for sent in train_list:
      sentence = nltk.word_tokenize(sent)
      count = len(sentence)

      if max_length < count:
        max_length = count

    return max_length

print(max_length(train))

Preprocessing the training data...
Preprocessing the validation data...
24


## Finetuning RoBERTa/BERT

### Change hyperparameters of the transformer

In [9]:
trainer_args = {
   'model_type':  'roberta',
   'model_name': 'roberta-large',
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',
   'fp16': True,
   'fp16_opt_level': 'O1',
   'max_seq_length': 30,
   'train_batch_size': 8,
   'eval_batch_size':8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 8,
   'weight_decay': 0,
   'learning_rate': 1.5e-5,
   'adam_epsilon': 0.5e-8, #0.01 = 1 * 10^-2 = 1e-2
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,
   'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,
   'overwrite_output_dir': True,
   'reprocess_input_data': False,
}

- Train

In [10]:
#10 min
def train_eval(train_df, valid_df, n_submission):

  model = ClassificationModel(
      'roberta', 
      'roberta-large', 
      args = trainer_args
      )
  
  print("training the model...")

  model.train_model(train)

  print("model trained!")
  print(" ")
  print("running predictions...")
  predictions, raw_outputs = model.predict(valid_df)

  submission_csv = pd.DataFrame(predictions, columns = ['LABEL'])

  submission_csv.to_csv (f'submission_{n_submission}.csv', index=True, header=True)

train_eval(train, valid, '3_4')

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classi

training the model...


/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

Running Epoch 0 of 8:   0%|          | 0/250 [00:00<?, ?it/s]

...

Running Epoch 6 of 8:   0%|          | 0/250 [00:00<?, ?it/s]

Running Epoch 7 of 8:   0%|          | 0/250 [00:00<?, ?it/s]

model trained!
 
running predictions...


  0%|          | 0/11000 [00:00<?, ?it/s]

  0%|          | 0/1375 [00:00<?, ?it/s]